In [1]:
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

In [2]:
from db_interaction import Base, Users, Events, Locations, Departments, Jobs, create_engine

In [3]:
engine = create_engine("sqlite:///demo_database.db", echo=True)

In [4]:
Base.metadata.create_all(engine)

2024-07-20 12:24:41,835 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-20 12:24:41,838 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-07-20 12:24:41,839 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-20 12:24:41,842 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2024-07-20 12:24:41,843 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-20 12:24:41,845 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("events")
2024-07-20 12:24:41,845 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-20 12:24:41,847 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("events")
2024-07-20 12:24:41,848 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-20 12:24:41,849 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("images")
2024-07-20 12:24:41,851 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-20 12:24:41,852 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("images")
2024-07-20 12:24:41,854 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07

In [5]:
vec_db_client = QdrantClient(url="http://localhost:6333")

In [6]:
COLLECTION_NAME = "faces"

if not vec_db_client.collection_exists(COLLECTION_NAME):
    vec_db_client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=512, distance=Distance.COSINE),
    )

In [7]:
with Session(engine) as session:
    user_list = [
        {
            "name": "Руслан",
            "surname": "Хаирбеков",
            "lastname": "Чеченец",
            "age": 15,
            "department_id": 1,
            "job_id": 2,
            "last_seen": 1721443298.6683102
        },
        {
            "name": "Мизеахмед",
            "surname": "бабахов",
            "lastname": "ывапролдж",
            "age": 17,
            "department_id": 4,
            "job_id": 34,
            "last_seen": 1721450505.2586892
        },
        {
            "name": "длптшз",
            "surname": "аапрапт",
            "lastname": "ыыыыыыыыы",
            "age": 1,
            "department_id": 15,
            "job_id": 26,
            "last_seen": 1721457711.3734472
        }
    ]

    user_vectors = [
        np.random.rand(512).tolist(),
        np.random.rand(512).tolist(),
        np.random.rand(512).tolist()
    ]

    for idx, data in enumerate(user_list):
        obj = Users(id=idx + 1, **data)
        session.add(obj)

    vec_db_client.upsert(
        collection_name="faces",
        points=[
            PointStruct(
                id=idx + 1,
                vector=vector,
            )
            for idx, vector in enumerate(user_vectors)
        ]
    )

    event_list = [
        {
            "user_id": 3,
            "location_id": 1,
            "timestamp": 1721457649.5150905
        },
        {
            "user_id": 2,
            "location_id": 2,
            "timestamp": 1721457649.5150905 + 3600 / 2
        },
        {
            "user_id": 1,
            "location_id": 3,
            "timestamp": 1721457649.5150905 + 3600
        }
    ]

    for idx, data in enumerate(event_list):
        obj = Events(id=idx + 1, **data)
        session.add(obj)

    location_list = [
        {
            "title": "Столовая"
        },
        {
            "title": "Офис"
        },
        {
            "title": "Цех"
        }
    ]

    for idx, data in enumerate(location_list):
        obj = Locations(id=idx + 1, **data)
        session.add(obj)

    department_list = [
        {
            "title": "Advetrisement"
        },
        {
            "title": "HR"
        },
        {
            "title": "Разработка"
        }
    ]

    for idx, data in enumerate(department_list):
        obj = Departments(id=idx + 1, **data)
        session.add(obj)

    job_list = [
        {
            "title": "Преподаватель"
        },
        {
            "title": "Директор"
        },
        {
            "title": "Работяга"
        },
    ]

    for idx, data in enumerate(job_list):
        obj = Jobs(id=idx + 1, **data)
        session.add(obj)
    
    session.commit()

2024-07-20 12:24:44,261 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-20 12:24:44,269 INFO sqlalchemy.engine.Engine INSERT INTO departments (id, title) VALUES (?, ?)
2024-07-20 12:24:44,273 INFO sqlalchemy.engine.Engine [generated in 0.00514s] [(1, 'Advetrisement'), (2, 'HR'), (3, 'Разработка')]
2024-07-20 12:24:44,284 INFO sqlalchemy.engine.Engine INSERT INTO events (id, user_id, location_id, timestamp) VALUES (?, ?, ?, ?)
2024-07-20 12:24:44,287 INFO sqlalchemy.engine.Engine [generated in 0.00398s] [(1, 3, 1, 1721457649.5150905), (2, 2, 2, 1721459449.5150905), (3, 1, 3, 1721461249.5150905)]
2024-07-20 12:24:44,299 INFO sqlalchemy.engine.Engine INSERT INTO jobs (id, title) VALUES (?, ?)
2024-07-20 12:24:44,308 INFO sqlalchemy.engine.Engine [generated in 0.01091s] [(1, 'Преподаватель'), (2, 'Директор'), (3, 'Работяга')]
2024-07-20 12:24:44,318 INFO sqlalchemy.engine.Engine INSERT INTO locations (id, title) VALUES (?, ?)
2024-07-20 12:24:44,321 INFO sqlalchemy.engine.Engine [ge